<h1>TCD ML Comp. 2019/20 - Rec Alg Click Pred (Group)</h1>
Importing required modules

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

URL of the datasets and storing datasets in dataframes

In [2]:
url_for_traindata =  os.path.join(os.path.realpath('./DATA/'),'tcdml1920-rec-click-pred--training.csv')
url_for_testdata = os.path.join(os.path.realpath('./DATA/'),'tcdml1920-rec-click-pred--test.csv')
output_url = os.path.join(os.path.realpath('./DATA/'),'output.csv')

training_dataframe = pd.read_csv(url_for_traindata) 
test_dataframe = pd.read_csv(url_for_testdata)

/Users/oommenkuruvilla/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<h3>Understanding taining and test data</h3> 

In [3]:
print(training_dataframe.isnull().sum())
print(training_dataframe.shape)
print(training_dataframe.head(2))
print(training_dataframe.tail(2))
print(test_dataframe.isnull().sum())
print(test_dataframe.shape)
print(test_dataframe.head(2))
print(test_dataframe.tail(2))

recommendation_set_id                0
user_id                              0
session_id                           0
query_identifier                     0
query_word_count                     0
query_char_count                     0
query_detected_language              0
query_document_id                    0
document_language_provided           0
year_published                       0
number_of_authors                    0
abstract_word_count                  0
abstract_char_count                  0
abstract_detected_language          78
first_author_id                      0
num_pubs_by_first_author             0
organization_id                      0
application_type                     0
item_type                            0
request_received                     0
hour_request_received                0
response_delivered                   0
rec_processing_time                  0
app_version                          2
app_lang                            80
user_os                  

From the data above, it seems there is around 1647 rows in test data with recommendation_set_id value as null  in dataset, which should not happen so removing it and also remvoing rows having query_char_count value as '\N'  and rec_processing_time > 100.

In [4]:
test_dataframe = test_dataframe.dropna(subset=['recommendation_set_id'])
index_to_drop = training_dataframe[(training_dataframe.query_char_count == '\\N') | (training_dataframe.rec_processing_time > 100)].index
training_dataframe.drop(index_to_drop,inplace= True)

<h3>Data pre-processing</h3>
  * Combining both training and test data for data preprocessing and also replacing '\N' with  nan.
  * Removing columns having more than 75% null value as it does

In [5]:
y = training_dataframe['set_clicked']
train_model = training_dataframe.drop(columns =['set_clicked'])
test_model =  test_dataframe.drop(columns =['set_clicked'])                                                     
combined_df = pd.concat([train_model, test_model])
combined_df.replace('\\N',np.nan,inplace = True)
print(train_model.shape)
print(test_model.shape)
combined_df.isnull().sum()
coulmns_to_drop_based_on_null = (combined_df.isnull().sum()[combined_df.isnull().sum() > 296600]).index.tolist()
combined_df.drop(columns = coulmns_to_drop_based_on_null,inplace= True)
print(combined_df.shape)

(385518, 45)
(9145, 45)
(394663, 31)


* Removing columns that are not useful based on doucmentation, few columns such as ctr are having no values in test file and based on trial and error.
* Columns of object type having null values updated with most frequent value.
* Changing columns having DtypeWarning(Columns (4,5,34) have mixed types) to numeric.

In [6]:
columns_not_useful = ['request_received', 'response_delivered', 'rec_processing_time', 'query_identifier', 'query_document_id', 'abstract_word_count', 'abstract_char_count', 'abstract_detected_language', 'number_of_recs_in_set', 'ctr', 'clicks', 'organization_id', 'local_time_of_request', 'timezone_by_ip']
combined_df.drop(columns = columns_not_useful,inplace= True)
print(combined_df.shape)
print(combined_df.isnull().sum())
object_type = combined_df.select_dtypes(include = ["object"]).columns
for item in object_type:
    most_frequent = combined_df[item].value_counts().index[0]
    combined_df[item] = combined_df[item].fillna(most_frequent)
combined_df['query_word_count'] = pd.to_numeric(combined_df['query_word_count'])
combined_df['query_char_count'] = pd.to_numeric(combined_df['query_char_count'])
combined_df['local_hour_of_request'] = pd.to_numeric(combined_df['local_hour_of_request'])
combined_df['recommendation_algorithm_id_used'] = pd.to_numeric(combined_df['recommendation_algorithm_id_used'])
print(combined_df.isnull().sum())

(394663, 17)
recommendation_set_id                    0
query_word_count                         2
query_char_count                         2
query_detected_language               3496
application_type                         0
item_type                            36972
hour_request_received                    0
app_version                         119342
app_lang                             17510
country_by_ip                         1635
local_hour_of_request                86452
recommendation_algorithm_id_used     10906
algorithm_class                          0
cbf_parser                          113252
search_title                             0
search_keywords                          0
search_abstract                          0
dtype: int64
recommendation_set_id               0
query_word_count                    0
query_char_count                    0
query_detected_language             0
application_type                    0
item_type                           0
hour_request_re

* Replacing '0' in application_type column to 'blog'
* Applying label encoding and one-hot encoding on categorical data.

In [7]:
combined_df.application_type.where(combined_df.application_type != '0','blog',inplace = True)
combined_df_onehot_encoded = pd.get_dummies(combined_df, columns =['cbf_parser','algorithm_class','query_detected_language','search_title', 'search_keywords', 'search_abstract','application_type'])
labelencoder = preprocessing.LabelEncoder()
cols = ['item_type', 'app_version', 'app_lang', 'country_by_ip']
for col in cols:
    combined_df_onehot_encoded[col] =  labelencoder.fit_transform(combined_df_onehot_encoded[col])
combined_df_onehot_encoded.shape

(394663, 67)

Splitting training and test Data.

In [8]:
y = y.astype(bool)
train_X = combined_df_onehot_encoded.iloc[0:385518, :]
test_X  = combined_df_onehot_encoded.iloc[385518:, :]

<h3>Randomo forest algorithm implemtation using  Scikit RandomForestRegressor</h3>

In [9]:
reg = RandomForestClassifier()
reg.fit(train_X,y)
prediction = reg.predict(test_X)
result = pd.DataFrame({'recommendation_set_id':test_dataframe.recommendation_set_id,'set_clicked':prediction.astype(int)})
result.to_csv(output_url, index = False)

/Users/oommenkuruvilla/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [10]:
# from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.ensemble import RandomForestClassifier
# from sklearn import svm, tree
# import xgboost
# classifiers = []
# model1 = xgboost.XGBClassifier()
# classifiers.append(model1)
# #model2 = svm.SVC()
# #classifiers.append(model2)
# model3 = tree.DecisionTreeClassifier()
# classifiers.append(model3)
# model4 = RandomForestClassifier()
# classifiers.append(model4)

# for clf in classifiers:
#     clf.fit(X_train, y_train)
#     y_pred= clf.predict(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     print("Accuracy is %s"%(acc))
#     #cm = confusion_matrix(y_test, y_pred)
#     #print("Confusion Matrix is %s"%(cm))
